VGG16 Modeli: vgg16_base, ImageNet veri seti üzerinde önceden eğitilmiş bir VGG16 modelini yüklüyor. Bu modelin son katmanları çıkarılıyor ve sadece öz nitelik çıkarımı için kullanılıyor. Tüm VGG16 katmanları donduruluyor, yani eğitilebilir değil.

VGG16 Çıkışı: VGG16 modelinden çıkan özellik haritası, ResNet50'nin giriş boyutuna (224x224) uygun hale getirilmek için UpSampling2D katmanıyla yeniden boyutlandırılıyor. Sonrasında Conv2D katmanıyla kanal sayısı 3'e indiriliyor.

ResNet50 Modeli: resnet50_base, ResNet50 modelini yükler ve bu modele, VGG16'dan gelen çıktı beslenir. ResNet50'nin son katmanları çıkarılarak, model öz nitelik çıkarımı için kullanılır.

Fully Connected Katmanlar: ResNet50'nin çıkışı GlobalAveragePooling2D katmanıyla düzleştirilir. Daha sonra iki adet fully connected (tam bağlantılı) katman eklenir (Dense katmanları) ve son olarak 10 sınıflı sınıflandırma yapabilen softmax aktivasyon fonksiyonuna sahip bir çıkış katmanı eklenir.

Model Derleme: Hibrit model, Adam optimizasyon algoritması ve categorical_crossentropy kayıp fonksiyonu ile derlenir. Eğitim sırasında doğruluk metriği takip edilecektir.

Model Özeti: hybrid_model.summary() fonksiyonu, modelin katmanlarının ve parametrelerinin detaylı bir özetini verir.

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, UpSampling2D, Conv2D
from tensorflow.keras.models import Model

#VGG16'nın ilk katmanları (ImageNet ile önceden eğitilmiş model)
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

#ResNet50'nin üst katmanları (ImageNet ile önceden eğitilmiş model)
resnet50_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

#VGG16'nın belirli katmanlarını dondurup özellik çıkarımları için kullanımlar
for layer in vgg16_base.layers:
    layer.trainable = False

#VGG16 modelinin çıktısı
vgg_output = vgg16_base.output

#Upsample the output of VGG16 to match ResNet50's input shape
vgg_output = UpSampling2D(size=(32, 32))(vgg_output) # Upsample to 224x224

#Add a 1x1 convolutional layer to reduce the number of channels to 3
vgg_output = Conv2D(3, (1, 1), activation='relu')(vgg_output)

#ResNet50'nin residual blokları
resnet_input = vgg_output
resnet_output = resnet50_base(resnet_input)

#Flatten ile özellikleri düzleştirip fully connected katmanlar ekleme
x = GlobalAveragePooling2D()(resnet_output)  # ResNet50'den gelen çıkışı düzleştir
x = Dense(1024, activation='relu')(x)  # Ekstra bir fully connected katman
x = Dense(512, activation='relu')(x)
output = Dense(10, activation='softmax')(x)  # Son katman, 10 sınıflı sınıflandırma

#Hibrit model
hybrid_model = Model(inputs=vgg16_base.input, outputs=output)

#Modeli derleme
hybrid_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#Modelin özeti
hybrid_model.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d (UpSampling2D)         │ (None, 224, 224, 512)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 224, 224, 3)         │           1,539 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 40,932,045 (156.14 MB)

 Trainable params: 26,164,237 (99.81 MB)

 Non-trainable params: 14,767,808 (56.33 MB)

Avantajlar:<br>
İki Güçlü Modelin Birleştirilmesi: VGG16 ve ResNet50, derin öğrenme topluluğunda kanıtlanmış, güçlü özellik çıkarıcılarıdır. Bu kod, iki modelin gücünü birleştirerek daha zengin özellik haritaları çıkarabilir, bu da modelin sınıflandırma performansını artırabilir.

Önceden Eğitilmiş Modellerin Kullanımı: VGG16 ve ResNet50, ImageNet veri seti üzerinde önceden eğitildiği için model sıfırdan eğitilmeye gerek duymaz. Bu, eğitimi hızlandırır ve daha az veriyle yüksek performans elde edilmesini sağlar.

Transfer Öğrenme: VGG16'nın katmanları dondurularak transfer öğrenme uygulanıyor. Bu, büyük veri setlerine ihtiyaç duymadan yeni bir veri seti üzerinde iyi performans elde edilmesine yardımcı olabilir.

Yeniden Boyutlandırma ve Kanal Düşürme: VGG16'nın çıkışı ResNet50'ye uygun hale getirilmek için UpSampling2D ve Conv2D katmanları kullanılıyor. Bu, iki modelin farklı yapılarını uyumlu hale getirir ve hibrit bir yapıya olanak tanır.

Özelleştirilebilir Fully Connected Katmanlar: Modelin sonunda eklenen fully connected katmanlar, modelin ihtiyaçlarına göre özelleştirilebilir. Bu katmanlar, daha karmaşık ve güçlü bir karar verme mekanizması sağlar.

Dezavantajlar:<br>
Modelin Hesaplama Yükü ve Bellek Kullanımı: VGG16 ve ResNet50 gibi büyük modellerin bir arada kullanılması, hesaplama açısından oldukça maliyetlidir. Özellikle büyük veri setlerinde eğitilirken, eğitim süresi uzun olabilir ve GPU/CPU kaynakları zorlanabilir. Ayrıca, modelin bellek gereksinimleri yüksektir.

Aşırı Uyum (Overfitting) Riski: Büyük ve karmaşık bir model olduğu için küçük veri setlerinde aşırı uyum riski taşıyabilir. Model, eğitim verisine fazla uyum sağlayabilir ve genelleme kabiliyeti azalabilir.

Giriş Boyutunun Katılığı: Hem VGG16 hem de ResNet50’nin sabit giriş boyutlarına (224x224) ihtiyaç duyması, giriş verilerini ön işlemden geçirmenizi gerektirir. Bu, bazı veri setlerinde esneklik kaybına neden olabilir ve modelin farklı giriş boyutlarına adapte olması zor olabilir.

İki Modelin Uyumu: VGG16'dan çıkan özellik haritası ResNet50'ye besleniyor. Ancak bu tür hibrit yaklaşımlar, teorik olarak mantıklı olsa da pratikte performans artışı garanti etmez. Özellikle, iki modelin birleştirilmesinde uyum sorunları olabilir ve bu model, performans açısından daha karmaşık hale gelebilir.

Eğitilebilir Katmanların Azalması: VGG16 katmanlarının dondurulması, eğitilebilir parametrelerin azaltılması anlamına gelir. Bu durum, modelin sadece son fully connected katmanları üzerinde öğrenme yapacağı anlamına gelir ve sınırlı veri durumunda faydalı olsa da büyük veri setlerinde modelin potansiyelini sınırlayabilir.

Kodun Amacı:
U-Net ve ResNet50 mimarilerini bir araya getirerek, segmentasyon görevleri için kullanılacak bir model oluşturmak.
Giriş görüntüsüne karşılık çıkış olarak bir segmentasyon maskesi üretir.
Adımlar:
ResNet50'nin Yüklenmesi:

ResNet50 modeli imagenet veri seti ile önceden eğitilmiş olarak yüklenir ve U-Net'in encoder kısmı olarak kullanılır.
Modelin tam bağlantı (fully connected) katmanları dahil edilmez (include_top=False).
Skip Connection Katmanları:

ResNet50'nin bazı katmanları (conv1_relu, conv2_block3_out, conv3_block4_out, conv4_block6_out) skip connection olarak kaydedilir. Bu katmanlar, U-Net'in decoder kısmında kullanılmak üzere saklanır.<br>
Encoder Çıkışı:<br>

ResNet50'nin son katmanı olan conv5_block3_out katmanı encoder kısmının çıktısı olarak kullanılır. Bu çıkış, segmentasyon işlemi için temel özellikleri içerir.<br>
Decoder Kısmı (Upsampling):<br>

Encoder'dan elde edilen özellik haritaları, Conv2DTranspose katmanları ile yukarı örneklenir (upsampling). Her yukarı örnekleme işleminden sonra, encoder'dan gelen skip connection'lar ile concatenate edilerek birleştirilir.
Yukarı örnekleme işlemleri dört aşamalı olup, her aşamada birleştirilen veriler üzerine bir dizi Conv2D işlemi uygulanır. Bu işlemler:<br>
16x16 -> 32x32<br>
32x32 -> 64x64<br>
64x64 -> 128x128<br>
128x128 -> 256x256 boyutlarında yapılır.<br>
Çıkış Katmanı:

En son katmanda, bir 1x1 Conv2D katmanı ile segmentasyon maskesi üretilir. Maskenin çıktısı, her piksel için bir değer sağlayan sigmoid aktivasyon fonksiyonuna sahiptir (0 ile 1 arasında değer).<br>
Modelin Oluşturulması:

Girdi olarak ResNet50'nin girişini kullanarak, çıktıda segmentasyon maskesi üreten bir U-Net modeli oluşturulmuştur.<br>
Modelin Derlenmesi:

Model, Adam optimizasyon algoritması ile binary_crossentropy kayıp fonksiyonunu kullanarak derlenir. Bu, genellikle ikili (binary) segmentasyon problemleri için uygun bir kayıp fonksiyonudur.
accuracy metriği, modelin performansını takip etmek için kullanılır.<br>
Model Özeti:

model.summary() fonksiyonu, modelin katmanlarını ve parametrelerini özetler.

In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, Model

def unet_resnet50(input_shape=(256, 256, 3)):
    # ResNet-50 modelini yükleyelim ve önceden eğitilmiş ağırlıkları kullanmayalım.
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

    # ResNet-50'den skip connection katmanlarını alalım
    skip1 = base_model.get_layer('conv1_relu').output   # 256x256
    skip2 = base_model.get_layer('conv2_block3_out').output  # 128x128
    skip3 = base_model.get_layer('conv3_block4_out').output  # 64x64
    skip4 = base_model.get_layer('conv4_block6_out').output  # 32x32

    # Encoder'den elde edilen en son katmanı alalım
    encoder_output = base_model.get_layer('conv5_block3_out').output  # 16x16

    # Decoder kısmı: U-Net'in yukarı örnekleme (upsampling) katmanları
    upsample1 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(encoder_output)  # 16x16 -> 32x32
    concat1 = layers.Concatenate()([upsample1, skip4])
    decode1 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(concat1)
    decode1 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(decode1)

    upsample2 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(decode1)  # 32x32 -> 64x64
    concat2 = layers.Concatenate()([upsample2, skip3])
    decode2 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(concat2)
    decode2 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(decode2)

    upsample3 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(decode2)  # 64x64 -> 128x128
    concat3 = layers.Concatenate()([upsample3, skip2])
    decode3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(concat3)
    decode3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(decode3)

    upsample4 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(decode3)  # 128x128 -> 256x256
    concat4 = layers.Concatenate()([upsample4, skip1])
    decode4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(concat4)
    decode4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(decode4)

    # Son katman: Bir segmentasyon maskesi üretmek için
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(decode4)  # Çıktı (256x256x1)

    # Modeli oluştur
    model = Model(inputs=base_model.input, outputs=outputs)

    return model

# Modeli derleyelim
model = unet_resnet50(input_shape=(256, 256, 3))
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Modelin özetini (summary) alalım
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 256, 256, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 262, 262, 3)    │              0 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 128, 128, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 128, 128, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 128, 128, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 130, 130, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 64, 64, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 64, 64, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 64, 64, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 64, 64, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 64, 64, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 64, 64, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 64, 64, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 64, 64, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 64, 64, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 40,969,985 (156.29 MB)

 Trainable params: 40,916,865 (156.09 MB)

 Non-trainable params: 53,120 (207.50 KB)

#Avantajlar:<br>
Pre-trained ResNet50 Kullanımı:

ResNet50, büyük bir veri seti olan ImageNet üzerinde önceden eğitildiği için görüntü tabanlı özellik çıkarımında oldukça başarılıdır. Bu, modelin güçlü bir encoder kısmına sahip olmasını sağlar.
Özellik çıkarımı açısından yüksek performans gösterir ve görsellerin detaylı özelliklerini daha iyi öğrenir.<br>
Skip Connections ile Bilgi Koruma:

U-Net mimarisi, her upsampling (yukarı örnekleme) adımında encoder'daki düşük seviyeli bilgileri skip connections ile kullanır. Bu, modelin hem düşük seviyeli hem de yüksek seviyeli bilgileri birleştirerek daha iyi segmentasyon yapmasını sağlar.
Skip connections, özelliklerin kaybolmaması ve daha iyi segmentasyon sonuçları alınması için önemlidir.<br>
Detaylı Segmentasyon:

U-Net, özellikle tıbbi görüntüleme, biyomedikal ve uydu görüntülerinde yaygın olarak kullanılan bir modeldir. ResNet50 ile birleştiğinde, daha karmaşık ve detaylı segmentasyon görevlerinde etkili olur.
Yüksek çözünürlüklü çıktı üretir, böylece segmentasyon maskesi detayları kaybetmez.<br>
Modüler ve Esnek Mimari:

Model, farklı boyutlardaki görüntüleri işleyebilir ve giriş boyutları kolayca değiştirilebilir.
Kullanıcı, skip connection ve upsampling katmanları ile oynayarak mimaride çeşitli özelleştirmeler yapabilir.<br>
Kapsamlı Feature Extraction:

ResNet50 ile daha derin ve karmaşık özellikler çıkarılır. Bu sayede model, daha zengin ve çok yönlü özellik haritalarına sahip olur.<br>
#Dezavantajlar:<br>
Büyük Model ve Bellek Kullanımı:

ResNet50, oldukça büyük bir modeldir ve çok sayıda parametre içerir. Bu, modelin eğitimi ve tahmin işlemi sırasında yüksek miktarda bellek (RAM/VRAM) kullanmasına sebep olur.
Aynı şekilde, bu model mobil cihazlar gibi düşük güçteki cihazlarda çalıştırılmak istendiğinde performans sorunları yaşanabilir.<br>
Yavaş Eğitim:

ResNet50'nin derin katmanları ve U-Net'in upsampling katmanları, eğitimi yavaşlatabilir. Daha derin ve karmaşık modeller, daha fazla eğitim süresi gerektirir.
Eğitim verilerinin boyutuna ve modelin parametre sayısına bağlı olarak modelin eğitimi zaman alıcı olabilir.<br>
Overfitting Riski:

Bu model, özellikle küçük veri setlerinde overfitting riski taşır. Modelin çok güçlü olması, eğitim verisine aşırı uyum sağlayarak genel genelleme kapasitesini düşürebilir.
Daha büyük veri seti gereksinimi olabilir.<br>
Pre-trained Modelin Sabit Ağırlıkları:

ResNet50'nin bazı katmanlarının dondurulmuş olması, modelin yeni veriler üzerinde tamamen özelleştirilmesini engelleyebilir. Model, bazı durumlarda veriyi tam olarak öğrenmeyebilir.
Veriyi daha iyi öğrenebilmesi için ağırlıkların yeniden eğitilebilir olması gerekebilir, bu da işlem süresini ve hesaplama maliyetini artırır.<br>
Tek Çıkışlı Segmentasyon Maskesi:

Çıkış katmanında sigmoid aktivasyon fonksiyonu ve tek kanal (1) kullanılmıştır. Bu, modelin sadece binary segmentasyon (ikili sınıflandırma) yapabileceği anlamına gelir. Çok sınıflı (multiclass) segmentasyon gereksinimi varsa, modelin yapısında değişiklik yapılması gerekir.